## Direct location with Sentinel-2 MSI sensor 

This notebook shows examples on direct location performed using MSI sensor and using DEM copernicus 90m and 30m with zarr format 


### Setup variables and configure S2MSIGeometry

In [ ]:
import os
import json
import logging

import folium
import numpy as np
from zarr.storage import FSStore

from tests.drivers.geojson_util import to_points
from tests.drivers.s2geo_legacy.s2geo_interface import S2geoInterface
from asgard.sensors.sentinel2.msi import S2MSIGeometry

In [ ]:
# these directories are required 
TEST_DIR = os.environ.get("TEST_DIR", "../tests")
ASGARD_DATA = os.environ.get("ASGARD_DATA", "/data/asgard")
DEM_90_20240605 = "S0__ADF_DEM90_20000101T000000_21000101T000000_20240605T132601.zarr"
DEM_30_20240604 = "S0__ADF_DEM30_20000101T000000_21000101T000000_20240604T233343.zarr"

In [ ]:
def setup_remote_dem(dem_filename) -> FSStore:
    """
    Create a FSStore pointing to a DEM stored remotely on S3
    """

    dem_path_s3 = f"simplecache::s3://common/ADFstatic/{dem_filename}"
    s3_config = {
        "key": os.environ.get("S3_COMMON_ACCESS"),
        "secret": os.environ.get("S3_COMMON_SECRET"),
        "client_kwargs": {
            "endpoint_url": "https://s3.sbg.perf.cloud.ovh.net",
            "region_name": "sbg",
        },
    }
    logging.info("Using DEM : %s", dem_path_s3)

    return FSStore(dem_path_s3, mode="r", s3=s3_config)


In [ ]:
# generating image coordinates
coords = np.zeros((100, 10, 2), dtype="int32")
for row in range(100):
    for col in range(10):
        coords[row, col, 0] = 50 * col
        coords[row, col, 1] = 20 * row


### Direct location with zarr DEM Copernicus 90m 

In [ ]:
# S2geo interface file -> Python dict
config = S2geoInterface(os.path.join(ASGARD_DATA, "S2MSIdataset/S2MSI_TDS1/L0c_DEM_zarr_S2GEO_Input_interface.xml")).read()
config["resources"].pop("geoid", "")
config["resources"].pop("dem_srtm", "")
config["resources"].pop("dem_globe", "")
# Configure DEM 90m Zarr
config["resources"]["dem_zarr"] = setup_remote_dem(DEM_90_20240605)
config["resources"]["geoid"] = os.path.join(
    ASGARD_DATA,
    "ADFstatic/S0__ADF_GEOI8_20000101T000000_21000101T000000_20240513T160103.zarr",
)

In [ ]:
# instantiation of an S2MSI product based on python config dict
msi_with_dem90 = S2MSIGeometry(**config)

In [ ]:
# running direct locations using a pre-generated coordinates grid
gnd_points_90m, _ = msi_with_dem90.direct_loc(coords, geometric_unit="B05/D11")

### Direct location with zarr DEM Copernicus 30m

In [ ]:
# Configure DEM 30m Zarr 
config["resources"]["dem_zarr"] = setup_remote_dem(DEM_30_20240604)

In [ ]:
# instantiation of an S2MSI product based on python config dict
msi_with_dem30 = S2MSIGeometry(**config)

In [ ]:
# running direct locations using a pre-generated coordinates grid
gnd_points_30m, _ = msi_with_dem30.direct_loc(coords, geometric_unit="B05/D10")

### Visualize direct location on map 

In [ ]:
# creating the GeoJSON file of data points
live_dir = os.path.join(TEST_DIR, "outputs/live")
if not os.path.exists(live_dir):
    os.makedirs(live_dir)

path_points90 = os.path.join(live_dir, "msi_points_90m.geojson")
path_points30 = os.path.join(live_dir, "msi_points_30m.geojson")

to_points(gnd_points_90m, path_points90)
to_points(gnd_points_30m, path_points30)

In [ ]:
# opening file and extracting coordinates
with open(path_points90, 'r') as f:
    json_data_point90 = json.load(f)
coordinates90 = json_data_point90['features'][0]['geometry']['coordinates']
with open(path_points30, 'r') as f:
    json_data_point30 = json.load(f)
coordinates30 = json_data_point30['features'][0]['geometry']['coordinates']

### Initialize folium map to visualize direct location points and tiles of zarr DEM

In [ ]:
# initializing the locations map
m = folium.Map(location=coordinates90[0], zoom_start=3)
# iterate through the coordinates and add them as circle-shaped markers to the map
for feature90 in coordinates90:
    coord90 = feature90[::-1]
    marker = folium.CircleMarker(location=coord90, radius=5, color='blue', fill=True, fill_color='blue')
    marker.add_to(m)

for feature30 in coordinates30:
    coordinates = feature30[::-1]
    marker = folium.CircleMarker(location=coordinates, radius=5, color='green', fill=True, fill_color='green')
    marker.add_to(m)

# saving the resulting map
m.save('map_msi_points.html')

In [ ]:
# plotting our GeoJSON locations 
m

### Add tiles on map

In [ ]:
# Create kw dict config in order to create rectangle to represent DEM tiles
kw = {
    "color": "black",
    "line_cap": "round",
    "fill": True,
    "fill_color": "blue",
    "weight": 5,
    "popup" : "tile90m_1"
}
bounds1_90m = np.rad2deg(np.array([[0.581776, -0.290888], [0.59632, -0.27634]]))
bounds2_90m = np.rad2deg(np.array([[0.56723, -0.290888], [0.58177, -0.27634]]))
bounds1_30m = np.rad2deg(np.array([[0.5817764, -0.290888], [0.58662455, -0.28604007]]))
bounds2_30m = np.rad2deg(np.array([[0.57208014, -0.2908882], [0.576928, -0.28604007]]))
bounds3_30m = np.rad2deg(np.array([[0.57692828, -0.2957363], [0.5817764, -0.2908882]]))
bounds4_30m = np.rad2deg(np.array([[0.57692828, -0.2908882], [0.5817764, -0.28604007]]))
bounds5_30m = np.rad2deg(np.array([[0.57208014, -0.2957363], [0.576928, -0.2908882]]))

folium.Rectangle(
    bounds=bounds1_90m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)


kw["popup"] = "tile90m_2"
folium.Rectangle(
    bounds=bounds2_90m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile30m_1"
kw["color"] = "green"
kw["fill_color"] = "yellow"
folium.Rectangle(
    bounds=bounds1_30m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile30m_2"
folium.Rectangle(
    bounds=bounds2_30m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile30m_3"
folium.Rectangle(
    bounds=bounds3_30m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

kw["popup"] = "tile30m_4"
folium.Rectangle(
    bounds=bounds4_30m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)


kw["popup"] = "tile30m_5"
folium.Rectangle(
    bounds=bounds5_30m,
    line_join="round",
    dash_array="1, 1",
    **kw,
).add_to(m)

# saving the resulting map
m.save('map_msi_points.html')

In [ ]:
# Visualize map
m